In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
os.chdir('/content/drive/MyDrive/통신망해커톤/Q2_data')
os.getcwd()

'/content/drive/MyDrive/통신망해커톤/Q2_data'

#데이터 불러오기

In [ ]:
train = pd.read_csv('Q2_train.csv')
test = pd.read_csv('Q2_test.csv')

In [ ]:
train

# 데이터 매핑

In [ ]:
#매핑한 csv 파일 불러와서 경보 메세지 1,2,3으로 매핑
map = pd.read_csv('msg매핑전체_v7.csv')
change_dict = dict(zip(map['msg'], map['type']))
train['alarmmsg_original'] = train['alarmmsg_original'].map(change_dict)
test['alarmmsg_original'] = test['alarmmsg_original'].map(change_dict)
# 범주형으로 변경
train['alarmmsg_original']=train['alarmmsg_original'].astype(str)
test['alarmmsg_original']=test['alarmmsg_original'].astype(str)

In [ ]:
test_df=test.copy()

#EDA

In [ ]:
#train에서 A제조사만 포함
#test에서는 A,B,C 제조사 모두 포함
print('train : ',train['root_cause_domain'].unique())
print('test : ',test['root_cause_domain'].unique())

In [ ]:
#train에서 slot과 port 변수에서 다량의 결측치가 발견됨
train.isna().sum()

In [ ]:
#test에서 slot과 port 변수에서 다량의 결측치가 발견됨
test.isna().sum()

In [ ]:
#test에서 site 변수의 고유값이 더 많은 것을 확인함
print('train : ',train['site'].nunique())
print('test : ',test['site'].nunique())

In [ ]:
#test에서 sysname 변수의 고유값이 더 많은 것을 확인함
print('train : ',train['sysname'].nunique())
print('test : ',test['sysname'].nunique())

In [ ]:
#root_cause_type 별 slot과 port 결측치 개수 시각화
plt.figure(figsize=(12, 6))
colors = ['green', 'gray', 'orange']

# 첫 번째 서브플롯: slotna
plt.subplot(1, 2, 1)
result_slotna = train[train['slot'].isnull()]['root_cause_type'].value_counts()
result_slotna.plot(kind='bar', color=colors)
plt.xlabel('Root Cause Type')
plt.ylabel('SlotNa')
plt.xticks(rotation=0)
plt.title('Number of SlotNa by root_cause_type')

# 두 번째 서브플롯: portna
colors = ['green', 'orange', 'gray']
plt.subplot(1, 2, 2)
result_portna = train[train['port'].isnull()]['root_cause_type'].value_counts()
result_portna.plot(kind='bar', color=colors)
plt.xlabel('Root Cause Type')
plt.ylabel('PortNa')
plt.xticks(rotation=0)
plt.title('Number of PortNa by root_cause_type')

plt.tight_layout()
plt.show()

# 데이터 전처리

## slot, port na

In [ ]:
# slot과 port 변수의 결측치 여부에 대한 파생변수 slotna, portna 생성
# train에 대해 진행
train['slotna'] = np.nan
train['portna'] = np.nan

train.loc[train[['slot']].isna( ).any(axis=1), 'slotna'] = 1
train.loc[train[['port']].isna( ).any(axis=1), 'portna'] = 1
train[['slotna', 'portna']] = train[['slotna', 'portna']].fillna(0)

train[['slotna', 'portna']] = train[['slotna', 'portna']].astype(int)
train[['slotna', 'portna']].value_counts()

slotna  portna
0       0         8717
1       1          357
0       1          240
1       0            8
dtype: int64

In [ ]:
# test에 대해서도 동일하게 진행
test['slotna'] = np.nan
test['portna'] = np.nan

test.loc[test[['slot']].isna( ).any(axis=1), 'slotna'] = 1
test.loc[test[['port']].isna( ).any(axis=1), 'portna'] = 1
test[['slotna', 'portna']] = test[['slotna', 'portna']].fillna(0)

test[['slotna', 'portna']] = test[['slotna', 'portna']].astype(int)
test[['slotna', 'portna']].value_counts()

slotna  portna
0       0         34853
1       1          2781
0       1            37
dtype: int64

## alarmmsg & timelen

In [ ]:
# alarmmsg의 1,2,3 개수를 세고 각각에 alarmlevel을 곱함
# alarmmsg = [1 2 1 3] , alarmlevel = [2,2,3,2] --> [alarmmsg1] = 5,[alarmmsg2] = 4,[alarmmsg3] = 6
msg = train[['ticketno', 'alarmlevel', 'alarmmsg_original']]
msg = pd.get_dummies(msg)
msg['alarmmsg_original_1'] = msg['alarmmsg_original_1'] * msg['alarmlevel']
msg['alarmmsg_original_2'] = msg['alarmmsg_original_2'] * msg['alarmlevel']
msg['alarmmsg_original_3'] = msg['alarmmsg_original_3'] * msg['alarmlevel']
msg.drop('alarmlevel', axis=1, inplace=True)
msg = msg.groupby('ticketno').sum()

In [ ]:
# ticketno별 경보가 마지막으로 발생한 시각까지 걸린 시간을 분 단위로 계산해 time_len이라는 파생변수 생성.
train['alarmtime']=pd.to_datetime(train['alarmtime'])
grouped = train.groupby('ticketno')['alarmtime'].agg(['max', 'min'])
grouped['time_len'] = grouped['max'] - grouped['min']
grouped['time_len'] = grouped['time_len'].dt.total_seconds() / 60
grouped.sort_values(by='ticketno',inplace=True)
grouped.reset_index(inplace=True)

In [ ]:
#train에서 ticketno를 기준으로 데이터 병합 및 결과 확인
train_merge = train[['ticketno', 'alarmlevel', 'slotna', 'portna','root_cause_type']]
train_merge.drop_duplicates(['ticketno'], inplace=True)

train_merge = pd.merge(msg, train_merge, on='ticketno')
train_merge=pd.concat([train_merge,grouped['time_len']],axis=1)
train_merge.head(1)

<ipython-input-16-ed8df3515fdb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_merge.drop_duplicates(['ticketno'], inplace=True)


In [ ]:
# test에 동일하게 진행
# alarmmsg * alarmlevel
msg = test[['ticketno', 'alarmlevel', 'alarmmsg_original']]
msg['alarmmsg_original'] = msg['alarmmsg_original']
msg = pd.get_dummies(msg)
msg['alarmmsg_original_1'] = msg['alarmmsg_original_1'] * msg['alarmlevel']
msg['alarmmsg_original_2'] = msg['alarmmsg_original_2'] * msg['alarmlevel']
msg['alarmmsg_original_3'] = msg['alarmmsg_original_3'] * msg['alarmlevel']
msg.drop('alarmlevel', axis=1, inplace=True)
msg = msg.groupby('ticketno').sum()

test = test[['ticketno', 'alarmlevel', 'slotna', 'portna']]
test.drop_duplicates(['ticketno'], inplace=True)
test = pd.merge(msg, test, on='ticketno')
test['ticketno'] = test['ticketno'].astype(int)

<ipython-input-18-01b1023ab7d2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msg['alarmmsg_original'] = msg['alarmmsg_original']
<ipython-input-18-01b1023ab7d2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop_duplicates(['ticketno'], inplace=True)


In [ ]:
# test에 동일하게 진행
# timelen
test_df['alarmtime']=pd.to_datetime(test_df['alarmtime'])
grouped = test_df.groupby('ticketno')['alarmtime'].agg(['max', 'min'])
grouped['time_len'] = grouped['max'] - grouped['min']
grouped['time_len'] = grouped['time_len'].dt.total_seconds() / 60
grouped.sort_values(by='ticketno',inplace=True)
grouped.reset_index(inplace=True)

In [ ]:
test=pd.concat([test,grouped['time_len']],axis=1)
test.head(1)

,ticketno,alarmmsg_original_1,alarmmsg_original_2,alarmmsg_original_3,alarmlevel,slotna,portna,time_len,time_len
0,15238899,14,8,0,4,1,1,0.0,0.0


## msglen

In [ ]:
# 베이스 라인 코드 사용

def ppr_data(df):
    # TODO: 데이터 전처리 코드 구현 ---------- #
    df = df.groupby('ticketno').apply(custom_info).reset_index()  # ticketno 기준 병합 및 전처리 수행

    x_df = df.iloc[:, :-1]
    y_df = df['root_cause_type']

    # ------------------------------------- #
    return x_df, y_df

def custom_info(group):  # 임의로 작성된 전처리 코드입니다.
    d = {}
    group.sort_values(by='alarmtime', ascending=True, inplace=True)  # 경보 순서 정렬
    d['alarmmsg_original'] = ' '.join(str(msg) for msg in group['alarmmsg_original'])  # 메시지 단순 병합
    if 'root_cause_type' in group.columns:  # 레이블 추출
        d['root_cause_type'] = group['root_cause_type'].iloc[0]  # 동일한 ticketno는 동일한 root_cause_type을 가짐
    else:
        d['root_cause_type'] = None  # 테스트 세트의 경우 정답 컬럼 없음
    return pd.Series(d, index=['alarmmsg_original', 'root_cause_type'])

train_df = train.copy()

x_train_df, y_train_df = ppr_data(train_df)
x_test_df, _ = ppr_data(test_df)  # 테스트 세트의 경우 정답 컬럼 없음

In [ ]:
#ticketno별 결합된 경보 메세지와 파생변수 추가된 train_merge 병합
#data에서 LinkCut, UnitFail, PowerFail을 1,2,3으로 변경
data = pd.concat([x_train_df,y_train_df,train_merge.drop(['ticketno','root_cause_type'],axis=1)],axis=1)
data['root_cause_type'] = data['root_cause_type'].apply(lambda x: 0 if x == 'LinkCut' else (1 if x == 'UnitFail' else (2 if x == 'PowerFail' else x)))

In [ ]:
#test도 동일한 방식으로 데이터 병합
test=pd.concat([x_test_df,test.drop(['ticketno'],axis=1)],axis=1)
test.head(1)

,ticketno,alarmmsg_original,alarmmsg_original_1,alarmmsg_original_2,alarmmsg_original_3,alarmlevel,slotna,portna,time_len,time_len
0,15238899.0,2 1 2 1,14,8,0,4,1,1,0.0,0.0


In [ ]:
#trian에서 1,2,3으로 매핑 후 ticketno별로 결합된 경보 메세지만 따로 list로 생성
length=data['alarmmsg_original']
length=list(length)

#경보 메세지 split 진행해서 각 전표 별 발생한 경보 메세지 개수 list에 담기
len_list=[]

for item in length:
    tokens=item.split()
    lengths=len(tokens)
    len_list.append(lengths)

#list 데이터 프레임으로 바꾸기
len_df=pd.DataFrame(len_list)
len_df.columns=['msg_len']

In [ ]:
#data에 msg_len 변수 추가 후 필요없는 변수 드랍
data=pd.concat([data, len_df],axis=1)
data.drop(['alarmmsg_original','alarmlevel'],axis=1,inplace=True)

In [ ]:
#test에서도 동일한 방식으로 msg_len 변수 생성 후 test_x에 결합
length=test['alarmmsg_original']
length=list(length)

len_list=[]

for item in length:
    tokens=item.split()
    lengths=len(tokens)
    len_list.append(lengths)

len_df=pd.DataFrame(len_list)
len_df.columns=['msg_len']

test_x=pd.concat([test, len_df],axis=1)
test_x.drop(['alarmmsg_original','alarmlevel'],axis=1,inplace=True)

In [ ]:
data.head(1)

,ticketno,root_cause_type,alarmmsg_original_1,alarmmsg_original_2,alarmmsg_original_3,slotna,portna,time_len,msg_len
0,14753084.0,2,21,13,0,1,1,5.016667,6


In [ ]:
test_x.head(1)

,ticketno,alarmmsg_original_1,alarmmsg_original_2,alarmmsg_original_3,slotna,portna,time_len,time_len,msg_len
0,15238899.0,14,8,0,1,1,0.0,0.0,4


In [ ]:
#파일로 만들기
data.to_csv('Q2_train_최종.csv',index=False)
test_x.to_csv('Q2_test_최종.csv',index=False)